# Pandas API on Spark

In [30]:
import pandas as pd
import pyspark.pandas as ps
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

## Conceptual foundations

### Configuration

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
spark.sql("set spark.sql.ansi.enabled=false")

DataFrame[key: string, value: string]

In [9]:
print("Spark version:", spark.version)
print("pyspark.pandas version:", ps.sys.version)

print(ps.options.display.max_rows)

Spark version: 4.0.0
pyspark.pandas version: 3.13.5 | packaged by conda-forge | (main, Jun 16 2025, 08:17:35) [GCC 13.3.0]
1000


## Cluster and data access

### Create simple dataframe

In [10]:
df = ps.DataFrame({"x": [1, 2, 3], "y": [10, 20, 30]})
print(type(df))
df.head()

<class 'pyspark.pandas.frame.DataFrame'>


,x,y
0,1,10
1,2,20
2,3,30


### Loading data

In [11]:
shark_incidents_dirty_df = ps.read_csv("shark-incidents.csv", index_col="UIN")
shark_incidents_df = ps.read_parquet("shark-incidents.parquet", index_col="uin")

shark_incidents_df.head()

,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,shark_common_name,shark_scientific_name,shark_identification_method,shark_identification_source,shark_length_m,basis_for_length,provoked_unprovoked,provocative_act,no_sharks,victim_activity,fish_speared_,commercial_dive_activity,object_of_bite,present_at_time_of_bite,direction_first_strike,shark_behaviour,victim_aware_of_shark,shark_captured,injury_location,injury_severity,victim_gender,victim_age,victim_clothing,clothing_coverage,dominant_clothing_colour,other_clothing_colour,clothing_pattern,fin_colour,diversionary_action_taken,diversionary_action_outcome,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,teeth_recovered,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,
uin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1791,fatal,NSW,near sydney,-33.866667,151.200000,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,NaN,swimming,None,None,None,None,None,None,None,None,torso,major lacerations,female,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"shark&survl, whitley 1958, book ref 1793",NaN
2,3,1803,injured,WA,"hamelin bay, faure island",-25.833333,113.883333,coastal,None,tiger shark,Galeocerdo cuvier,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,swam at victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,NaN,NaN,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"balgridge,green,taylor,whitley 1940",NaN
3,1,1807,injured,NSW,"cockle bay, sydney harbour",-33.866667,151.200000,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit victim on wrist,None,None,"arm, hand",minor lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,sydney gazette 18.1.1807,NaN
4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.800000,147.533333,coastal,None,None,None,None,None,NaN,None,provoked,None,1.0,swimming,None,None,None,catch,None,bit victim on leg,N,None,leg,major lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,NaN,100.0,None,None,None,None,NaN,None,None,witness account,"shark&survl, c. black researcher",NaN
5,1,1825,injured,NSW,"kirribili point, sydney harbour",-33.850000,151.216667,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit legs,None,None,leg,minor lacerations,male,15.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,maitland daily mercury 13.11.1899,NaN


### Creating dataframes programmatically

In [ ]:
df_spark = ps.DataFrame({"name": ["Alice", "Bob"], "age": [25, 32]})
df_spark.head()

In [ ]:
df_local = pd.DataFrame({"city": ["London", "Paris"], "country": ["UK", "FR"]})
df_from_pandas = ps.from_pandas(df_local)
df_from_pandas.head()

### Inspecting data

In [12]:
shark_incidents_df.head(5)

,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,shark_common_name,shark_scientific_name,shark_identification_method,shark_identification_source,shark_length_m,basis_for_length,provoked_unprovoked,provocative_act,no_sharks,victim_activity,fish_speared_,commercial_dive_activity,object_of_bite,present_at_time_of_bite,direction_first_strike,shark_behaviour,victim_aware_of_shark,shark_captured,injury_location,injury_severity,victim_gender,victim_age,victim_clothing,clothing_coverage,dominant_clothing_colour,other_clothing_colour,clothing_pattern,fin_colour,diversionary_action_taken,diversionary_action_outcome,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,teeth_recovered,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,
uin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1791,fatal,NSW,near sydney,-33.866667,151.200000,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,NaN,swimming,None,None,None,None,None,None,None,None,torso,major lacerations,female,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"shark&survl, whitley 1958, book ref 1793",NaN
2,3,1803,injured,WA,"hamelin bay, faure island",-25.833333,113.883333,coastal,None,tiger shark,Galeocerdo cuvier,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,swam at victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,NaN,NaN,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"balgridge,green,taylor,whitley 1940",NaN
3,1,1807,injured,NSW,"cockle bay, sydney harbour",-33.866667,151.200000,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit victim on wrist,None,None,"arm, hand",minor lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,sydney gazette 18.1.1807,NaN
4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.800000,147.533333,coastal,None,None,None,None,None,NaN,None,provoked,None,1.0,swimming,None,None,None,catch,None,bit victim on leg,N,None,leg,major lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,NaN,100.0,None,None,None,None,NaN,None,None,witness account,"shark&survl, c. black researcher",NaN
5,1,1825,injured,NSW,"kirribili point, sydney harbour",-33.850000,151.216667,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit legs,None,None,leg,minor lacerations,male,15.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,maitland daily mercury 13.11.1899,NaN


In [13]:
shark_incidents_df.info()

<class 'pyspark.pandas.frame.DataFrame'>
Index: 1196 entries, 1 to 1196
Data columns (total 59 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   incident_month               1196 non-null   int64  
 1   incident_year                1196 non-null   int64  
 2   victim_injury                1196 non-null   object 
 3   state                        1196 non-null   object 
 4   location                     1193 non-null   object 
 5   latitude                     1196 non-null   float64
 6   longitude                    1196 non-null   float64
 7   site_category                1196 non-null   object 
 8   site_category_comment        23 non-null     object 
 9   shark_common_name            1144 non-null   object 
 10  shark_scientific_name        1144 non-null   object 
 11  shark_identification_method  994 non-null    object 
 12  shark_identification_source  138 non-null    object 
 13  shark_length_m      

In [14]:
shark_incidents_df.describe()

,incident_month,incident_year,latitude,longitude,shark_length_m,no_sharks,victim_age,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,air_temperature_°c,
count,1196.000000,1196.000000,1196.000000,1196.000000,579.000000,1107.000000,699.000000,97.000000,83.000000,514.000000,518.000000,230.000000,91.000000,228.000000,69.000000,356.000000,40.000000,1.0
mean,5.915552,1966.895485,-28.584045,143.191359,2.698446,1.035230,28.164521,1.556701,3.289157,1281.745136,2.129923,59.370000,20.981319,5.961404,8.347826,3201.761236,24.175000,415438758.0
std,4.090449,48.260971,7.931424,13.194388,1.209384,0.347591,13.794858,1.561043,5.511695,410.957788,5.485205,253.578818,4.127090,9.286013,14.188672,21638.809550,4.914069,NaN
min,1.000000,1791.000000,-43.652300,96.816667,0.300000,1.000000,0.000000,0.000000,0.000000,130.000000,0.000000,0.100000,0.300000,0.500000,0.000000,1.000000,10.000000,415438758.0
25%,2.000000,1931.000000,-33.866667,141.600000,1.800000,1.000000,17.000000,1.000000,1.000000,930.000000,0.000000,3.000000,19.000000,1.000000,1.000000,30.000000,22.000000,415438758.0
50%,5.000000,1983.000000,-31.816667,149.983333,2.600000,1.000000,25.000000,1.000000,2.000000,1300.000000,0.000000,10.000000,21.000000,2.000000,5.000000,80.000000,25.000000,415438758.0
75%,10.000000,2010.000000,-24.383333,152.183333,3.500000,1.000000,36.000000,2.000000,3.000000,1620.000000,1.000000,30.000000,23.000000,7.000000,10.000000,200.000000,27.000000,415438758.0
max,12.000000,2022.000000,-9.400000,159.150000,6.000000,10.000000,84.000000,12.000000,40.000000,2330.000000,45.000000,2160.000000,29.000000,80.000000,100.000000,280000.000000,35.000000,415438758.0


## Transformations

### Selecting and filtering

In [15]:
subset_df = shark_incidents_df[["shark_common_name", "victim_injury"]]
subset_df.head()

,shark_common_name,victim_injury
uin,,
1,white shark,fatal
2,tiger shark,injured
3,bull shark,injured
4,None,fatal
5,bull shark,injured


In [16]:
fatal_df = subset_df[subset_df["victim_injury"] == "fatal"]
fatal_df.head()

,shark_common_name,victim_injury
uin,,
1,white shark,fatal
4,None,fatal
10,whaler shark,fatal
11,whaler shark,fatal
12,bull shark,fatal


### Column expressions

In [17]:
shark_incidents_df["was_fatal"] = shark_incidents_df["victim_injury"] == "fatal"
shark_incidents_df.head()

,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,shark_common_name,shark_scientific_name,shark_identification_method,shark_identification_source,shark_length_m,basis_for_length,provoked_unprovoked,provocative_act,no_sharks,victim_activity,fish_speared_,commercial_dive_activity,object_of_bite,present_at_time_of_bite,direction_first_strike,shark_behaviour,victim_aware_of_shark,shark_captured,injury_location,injury_severity,victim_gender,victim_age,victim_clothing,clothing_coverage,dominant_clothing_colour,other_clothing_colour,clothing_pattern,fin_colour,diversionary_action_taken,diversionary_action_outcome,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,teeth_recovered,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,,was_fatal
uin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1791,fatal,NSW,near sydney,-33.866667,151.200000,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,NaN,swimming,None,None,None,None,None,None,None,None,torso,major lacerations,female,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"shark&survl, whitley 1958, book ref 1793",NaN,True
2,3,1803,injured,WA,"hamelin bay, faure island",-25.833333,113.883333,coastal,None,tiger shark,Galeocerdo cuvier,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,swam at victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,NaN,NaN,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"balgridge,green,taylor,whitley 1940",NaN,False
3,1,1807,injured,NSW,"cockle bay, sydney harbour",-33.866667,151.200000,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit victim on wrist,None,None,"arm, hand",minor lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,sydney gazette 18.1.1807,NaN,False
4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.800000,147.533333,coastal,None,None,None,None,None,NaN,None,provoked,None,1.0,swimming,None,None,None,catch,None,bit victim on leg,N,None,leg,major lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,NaN,100.0,None,None,None,None,NaN,None,None,witness account,"shark&survl, c. black researcher",NaN,True
5,1,1825,injured,NSW,"kirribili point, sydney harbour",-33.850000,151.216667,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit legs,None,None,leg,minor lacerations,male,15.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,maitland daily mercury 13.11.1899,NaN,False


### Handling missing data

In [18]:
shark_incidents_dirty_df.tail()

,Incident.month,Incident.year,Victim.injury,State,Location,Latitude,Longitude,Site.category,Site.category.comment,Shark.common.name,Shark.scientific.name,Shark.identification.method,Shark.identification.source,Shark.length.m,Basis.for.length,Provoked/unprovoked,Provocative.act,No.sharks,Victim.activity,Fish.speared?,Commercial.dive.activity,Object.of.bite,Present.at.time.of.bite,Direction.first.strike,Shark.behaviour,Victim.aware.of.shark,Shark.captured,Injury.location,Injury.severity,Victim.gender,Victim.age,Victim.clothing,Clothing.coverage,Dominant.clothing.colour,Other.clothing.colour,Clothing.pattern,Fin.colour,Diversionary.action.taken,Diversionary.action.outcome,People <3m,People 3-15m,Time.of.incident,Depth.of.incident.m,Teeth.recovered,Time.in.water.min,Water.temperature.°C,Total.water.depth.m,Water.visability.m,Distance.to.shore.m,Spring.or.neap.tide,Tidal.cycle,Wind.condition,Weather.condition,Air.temperature.°C,Personal.protective.device,Deterrent.brand.and.type,Data.source,Reference,_c59
UIN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,None,None
None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,None,None
None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,None,None
None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,None,None
None,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,None,None


In [19]:
shark_incidents_dirty_df.isnull().sum(axis=0)

Incident.month                   11
Incident.year                    11
Victim.injury                    10
State                            10
Location                         13
Latitude                         10
Longitude                        10
Site.category                    10
Site.category.comment          1183
Shark.common.name                63
Shark.scientific.name            63
Shark.identification.method     213
Shark.identification.source    1069
Shark.length.m                  628
Basis.for.length                725
Provoked/unprovoked              15
Provocative.act                1055
No.sharks                       100
Victim.activity                  36
Fish.speared?                  1207
Commercial.dive.activity       1091
Object.of.bite                  921
Present.at.time.of.bite         617
Direction.first.strike          905
Shark.behaviour                 227
Victim.aware.of.shark           593
Shark.captured                 1139
Injury.location             

In [20]:
clean_df = shark_incidents_dirty_df.dropna(subset=["Shark.common.name"])
clean_df.tail()

,Incident.month,Incident.year,Victim.injury,State,Location,Latitude,Longitude,Site.category,Site.category.comment,Shark.common.name,Shark.scientific.name,Shark.identification.method,Shark.identification.source,Shark.length.m,Basis.for.length,Provoked/unprovoked,Provocative.act,No.sharks,Victim.activity,Fish.speared?,Commercial.dive.activity,Object.of.bite,Present.at.time.of.bite,Direction.first.strike,Shark.behaviour,Victim.aware.of.shark,Shark.captured,Injury.location,Injury.severity,Victim.gender,Victim.age,Victim.clothing,Clothing.coverage,Dominant.clothing.colour,Other.clothing.colour,Clothing.pattern,Fin.colour,Diversionary.action.taken,Diversionary.action.outcome,People <3m,People 3-15m,Time.of.incident,Depth.of.incident.m,Teeth.recovered,Time.in.water.min,Water.temperature.°C,Total.water.depth.m,Water.visability.m,Distance.to.shore.m,Spring.or.neap.tide,Tidal.cycle,Wind.condition,Weather.condition,Air.temperature.°C,Personal.protective.device,Deterrent.brand.and.type,Data.source,Reference,_c59
UIN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1191,2,2022,fatal,NSW,"little bay, sydney",-30.8830,153.0830,coastal,bay to open ocean,white shark,Carcharodon carcharias,None,government official,4.5,None,unprovoked,None,1,swimming,None,None,None,fishing,below,video online (graphic),N,N,None,None,male,35.0,None,None,None,None,None,None,None,None,None,NaN,1700.0,NaN,None,NaN,NaN,NaN,NaN,50.0,NaN,None,None,None,NaN,None,None,None,https://www.watoday.com.au/national/nsw/one-of...,None
1192,2,2022,injured,QLD,"Redcliffe, Brisbane",-27.2333,153.2667,ocean/pelagic,None,tiger shark,Galeocerdo cuvier,None,witness/victim,3.5,None,unprovoked,None,1,other:floating,None,None,None,fishing,None,None,None,None,leg,major lacerations,None,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,None,None
1193,2,2022,injured,WA,"wylie bay, esperance",-33.84848,121.9184,coastal,bay to open ocean,white shark,Carcharodon carcharias,None,government official,3.3,None,unprovoked,None,1,other:floating,None,None,None,None,None,None,None,None,torso,major lacerations,female,20.0,None,None,None,None,None,None,None,None,None,NaN,1200.0,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,https://www.dailymail.co.uk/news/article-10483...,None
1194,2,2022,injured,NSW,"shelley beach, manly",-33.8006,151.2982,coastal,bay to open ocean,wobbegong,Orectolobidae,victim,None,NaN,None,unprovoked,None,1,diving,None,None,None,None,None,None,None,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,None,None
1195,3,2022,injured,NSW,crowdy head beach,-32,152.7512,coastal,surf beach,wobbegong,Orectolobidae,None,government official,NaN,None,unprovoked,None,1,swimming,None,None,None,baitfish,None,None,None,None,left leg,minor lacerations,male,32.0,swimwear,None,black,None,None,None,None,None,None,NaN,1720.0,NaN,None,40.0,20.0,1.2,NaN,50.0,NaN,None,None,None,NaN,None,None,None,None,None


### Sorting and renaming

In [22]:
longest_df = shark_incidents_df.sort_values("shark_length_m", ascending=False)
longest_df.head()

,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,shark_common_name,shark_scientific_name,shark_identification_method,shark_identification_source,shark_length_m,basis_for_length,provoked_unprovoked,provocative_act,no_sharks,victim_activity,fish_speared_,commercial_dive_activity,object_of_bite,present_at_time_of_bite,direction_first_strike,shark_behaviour,victim_aware_of_shark,shark_captured,injury_location,injury_severity,victim_gender,victim_age,victim_clothing,clothing_coverage,dominant_clothing_colour,other_clothing_colour,clothing_pattern,fin_colour,diversionary_action_taken,diversionary_action_outcome,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,teeth_recovered,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,,was_fatal
uin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
345,2,1936,fatal,NSW,"manly, south steyne, sydney",-33.783333,151.283333,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,6.0,witness/victim,unprovoked,None,1,swimming,None,None,None,swimming,None,None,N,None,other: body not recovered,None,male,14.0,None,None,None,None,None,None,None,None,NaN,1.0,1500.0,0.0,None,NaN,23.0,2.0,NaN,100.0,None,None,None,None,NaN,None,None,media outlet,"sharpe, green, shark&survl, the canberra times...",NaN,True
596,2,1982,fatal,TAS,south cape bay,-43.566667,146.883333,coastal,None,white shark,Carcharodon carcharias,direct observation,None,6.0,"witness/victim, other: comparison to boat",provoked,None,1,spearfishing,None,None,None,"spearfishing, boating",None,bit victim on body,N,None,other: body not recovered,None,male,32.0,wetsuit/drysuit,full body,black,None,None,None,None,None,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,50.0,None,None,moderate breeze,None,NaN,speargun,None,media outlet,"green, shark&survl, c.black, daily mirror 1.3....",NaN,True
606,6,1984,uninjured,TAS,cape raoul,-43.233333,147.783333,coastal,None,white shark,Carcharodon carcharias,direct observation,None,6.0,witness/victim,provoked,enticed shark,1,diving,None,None,Catch,catch,None,swam towards victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,NaN,NaN,NaN,NaN,None,NaN,NaN,5.0,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,the mercury 16.6.1984,NaN,False
609,10,1984,uninjured,TAS,"the lanterns, fortescue bay",-43.133333,148.000000,coastal,None,white shark,Carcharodon carcharias,direct observation,None,6.0,witness/victim,provoked,enticed shark,1,diving,None,Y,Catch,catch,None,circled victim,Y,None,None,None,male,NaN,wetsuit/drysuit,full body,black,None,None,None,hid in rock cave,None,NaN,NaN,NaN,23.0,None,5.0,NaN,NaN,23.0,NaN,None,None,None,None,NaN,None,None,media outlet,the mercury 15.10.1984,NaN,False
613,3,1985,fatal,SA,"wiseman's beach, port lincoln",-34.466667,136.016667,coastal,None,white shark,Carcharodon carcharias,direct observation,None,6.0,witness/victim,unprovoked,None,1,snorkelling,None,None,None,snorkelling,behind,bit victim,N,None,torso,major lacerations,female,33.0,wetsuit/drysuit,full body,black,None,None,yellow,None,None,NaN,NaN,1230.0,NaN,None,NaN,NaN,NaN,NaN,150.0,None,None,None,None,NaN,None,None,media outlet,"papers, taylor.",NaN,True


In [23]:
renamed_df = shark_incidents_dirty_df.rename(columns={"Shark.common.name": "species"})
renamed_df.head()

,Incident.month,Incident.year,Victim.injury,State,Location,Latitude,Longitude,Site.category,Site.category.comment,species,Shark.scientific.name,Shark.identification.method,Shark.identification.source,Shark.length.m,Basis.for.length,Provoked/unprovoked,Provocative.act,No.sharks,Victim.activity,Fish.speared?,Commercial.dive.activity,Object.of.bite,Present.at.time.of.bite,Direction.first.strike,Shark.behaviour,Victim.aware.of.shark,Shark.captured,Injury.location,Injury.severity,Victim.gender,Victim.age,Victim.clothing,Clothing.coverage,Dominant.clothing.colour,Other.clothing.colour,Clothing.pattern,Fin.colour,Diversionary.action.taken,Diversionary.action.outcome,People <3m,People 3-15m,Time.of.incident,Depth.of.incident.m,Teeth.recovered,Time.in.water.min,Water.temperature.°C,Total.water.depth.m,Water.visability.m,Distance.to.shore.m,Spring.or.neap.tide,Tidal.cycle,Wind.condition,Weather.condition,Air.temperature.°C,Personal.protective.device,Deterrent.brand.and.type,Data.source,Reference,_c59
UIN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1791,fatal,NSW,near sydney,-33.86666667,151.2,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,NaN,swimming,None,None,None,None,None,None,None,None,torso,major lacerations,female,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,book,"shark&survl, whitley 1958, book ref 1793",None
2,3,1803,injured,WA,"hamelin bay, faure island",-25.83333333,113.8833333,coastal,None,tiger shark,Galeocerdo cuvier,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,swam at victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,None,NaN,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,book,"balgridge,green,taylor,whitley 1940",None
3,1,1807,injured,NSW,"cockle bay, sydney harbour",-33.86666667,151.2,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit victim on wrist,None,None,"arm, hand",minor lacerations,male,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,media outlet,sydney gazette 18.1.1807,None
4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.8,147.5333333,coastal,None,None,None,None,None,NaN,None,provoked,None,1.0,swimming,None,None,None,catch,None,bit victim on leg,N,None,leg,major lacerations,male,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,1.0,None,NaN,NaN,NaN,NaN,100.0,NaN,None,None,None,NaN,None,None,witness account,"shark&survl, c. black researcher",None
5,1,1825,injured,NSW,"kirribili point, sydney harbour",-33.85,151.2166667,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit legs,None,None,leg,minor lacerations,male,15.0,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,media outlet,maitland daily mercury 13.11.1899,None


## Aggregations, joins and dates

### GroupBy and aggregations

In [24]:
length_df = (
    shark_incidents_df.groupby("shark_common_name")
    .agg(largest=("shark_length_m", "max"), average=("shark_length_m", "median"))
    .sort_values(["average"], ascending=False)
)

length_df.head()

,largest,average
shark_common_name,,
white shark,6.0,3.0
tiger shark,5.0,3.0
hammerhead shark,4.0,3.0
broadnose sevengill shark,3.0,2.6
sevengill shark,2.6,2.6


### Joins

In [25]:
shark_incidents_df.merge(
    length_df, how="left", left_on="shark_common_name", right_index=True
).head()

,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,shark_common_name,shark_scientific_name,shark_identification_method,shark_identification_source,shark_length_m,basis_for_length,provoked_unprovoked,provocative_act,no_sharks,victim_activity,fish_speared_,commercial_dive_activity,object_of_bite,present_at_time_of_bite,direction_first_strike,shark_behaviour,victim_aware_of_shark,shark_captured,injury_location,injury_severity,victim_gender,victim_age,victim_clothing,clothing_coverage,dominant_clothing_colour,other_clothing_colour,clothing_pattern,fin_colour,diversionary_action_taken,diversionary_action_outcome,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,teeth_recovered,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,,was_fatal,largest,average
uin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1791,fatal,NSW,near sydney,-33.866667,151.200000,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,NaN,swimming,None,None,None,None,None,None,None,None,torso,major lacerations,female,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"shark&survl, whitley 1958, book ref 1793",NaN,True,6.0,3.00
2,3,1803,injured,WA,"hamelin bay, faure island",-25.833333,113.883333,coastal,None,tiger shark,Galeocerdo cuvier,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,swam at victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,NaN,NaN,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"balgridge,green,taylor,whitley 1940",NaN,False,5.0,3.00
3,1,1807,injured,NSW,"cockle bay, sydney harbour",-33.866667,151.200000,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit victim on wrist,None,None,"arm, hand",minor lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,sydney gazette 18.1.1807,NaN,False,4.0,2.25
4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.800000,147.533333,coastal,None,None,None,None,None,NaN,None,provoked,None,1.0,swimming,None,None,None,catch,None,bit victim on leg,N,None,leg,major lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,NaN,100.0,None,None,None,None,NaN,None,None,witness account,"shark&survl, c. black researcher",NaN,True,NaN,NaN
5,1,1825,injured,NSW,"kirribili point, sydney harbour",-33.850000,151.216667,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit legs,None,None,leg,minor lacerations,male,15.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,maitland daily mercury 13.11.1899,NaN,False,4.0,2.25


### Dates and times

In [26]:
shark_incidents_df["incident_day"] = 1

shark_incidents_df["incident_date"] = ps.to_datetime(
    shark_incidents_df.rename(
        columns={
            "incident_day": "day",
            "incident_month": "month",
            "incident_year": "year",
        }
    )
).head()

shark_incidents_df.head()

,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,shark_common_name,shark_scientific_name,shark_identification_method,shark_identification_source,shark_length_m,basis_for_length,provoked_unprovoked,provocative_act,no_sharks,victim_activity,fish_speared_,commercial_dive_activity,object_of_bite,present_at_time_of_bite,direction_first_strike,shark_behaviour,victim_aware_of_shark,shark_captured,injury_location,injury_severity,victim_gender,victim_age,victim_clothing,clothing_coverage,dominant_clothing_colour,other_clothing_colour,clothing_pattern,fin_colour,diversionary_action_taken,diversionary_action_outcome,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,teeth_recovered,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,,was_fatal,incident_day,incident_date
uin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1791,fatal,NSW,near sydney,-33.866667,151.200000,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,NaN,swimming,None,None,None,None,None,None,None,None,torso,major lacerations,female,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"shark&survl, whitley 1958, book ref 1793",NaN,True,1,1791-01-01
2,3,1803,injured,WA,"hamelin bay, faure island",-25.833333,113.883333,coastal,None,tiger shark,Galeocerdo cuvier,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,swam at victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,NaN,NaN,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"balgridge,green,taylor,whitley 1940",NaN,False,1,1803-03-01
3,1,1807,injured,NSW,"cockle bay, sydney harbour",-33.866667,151.200000,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit victim on wrist,None,None,"arm, hand",minor lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,sydney gazette 18.1.1807,NaN,False,1,1807-01-01
4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.800000,147.533333,coastal,None,None,None,None,None,NaN,None,provoked,None,1.0,swimming,None,None,None,catch,None,bit victim on leg,N,None,leg,major lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,NaN,100.0,None,None,None,None,NaN,None,None,witness account,"shark&survl, c. black researcher",NaN,True,1,1820-01-01
5,1,1825,injured,NSW,"kirribili point, sydney harbour",-33.850000,151.216667,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit legs,None,None,leg,minor lacerations,male,15.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,maitland daily mercury 13.11.1899,NaN,False,1,1825-01-01


In [28]:
shark_incidents_df.dtypes

incident_month             int64
incident_year              int64
victim_injury             object
state                     object
location                  object
                       ...      
reference                 object
                           int64
was_fatal                   bool
incident_day               int32
incident_date     datetime64[ns]
Length: 62, dtype: object

## Advanced operations

### Window functions

In [29]:
shark_incidents_df["length_rank"] = shark_incidents_df.groupby("shark_common_name")[
    "shark_length_m"
].rank(method="dense", ascending=False)

shark_incidents_df.head()

,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,shark_common_name,shark_scientific_name,shark_identification_method,shark_identification_source,shark_length_m,basis_for_length,provoked_unprovoked,provocative_act,no_sharks,victim_activity,fish_speared_,commercial_dive_activity,object_of_bite,present_at_time_of_bite,direction_first_strike,shark_behaviour,victim_aware_of_shark,shark_captured,injury_location,injury_severity,victim_gender,victim_age,victim_clothing,clothing_coverage,dominant_clothing_colour,other_clothing_colour,clothing_pattern,fin_colour,diversionary_action_taken,diversionary_action_outcome,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,teeth_recovered,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,,was_fatal,incident_day,incident_date,length_rank
uin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1791,fatal,NSW,near sydney,-33.866667,151.200000,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,NaN,swimming,None,None,None,None,None,None,None,None,torso,major lacerations,female,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"shark&survl, whitley 1958, book ref 1793",NaN,True,1,1791-01-01,39.0
2,3,1803,injured,WA,"hamelin bay, faure island",-25.833333,113.883333,coastal,None,tiger shark,Galeocerdo cuvier,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,swam at victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,NaN,NaN,NaN,0.0,None,1.0,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,book,"balgridge,green,taylor,whitley 1940",NaN,False,1,1803-03-01,25.0
3,1,1807,injured,NSW,"cockle bay, sydney harbour",-33.866667,151.200000,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit victim on wrist,None,None,"arm, hand",minor lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,sydney gazette 18.1.1807,NaN,False,1,1807-01-01,24.0
4,1,1820,fatal,TAS,"sweetwater point, pitt water",-42.800000,147.533333,coastal,None,None,None,None,None,NaN,None,provoked,None,1.0,swimming,None,None,None,catch,None,bit victim on leg,N,None,leg,major lacerations,male,NaN,None,None,None,None,None,None,None,None,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,NaN,100.0,None,None,None,None,NaN,None,None,witness account,"shark&survl, c. black researcher",NaN,True,1,1820-01-01,6.0
5,1,1825,injured,NSW,"kirribili point, sydney harbour",-33.850000,151.216667,estuary/harbour,None,bull shark,Carcharhinus leucas,"bite analysis, shark behaviour, geographical l...",None,NaN,None,unprovoked,None,1.0,swimming,None,None,None,None,None,bit legs,None,None,leg,minor lacerations,male,15.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,maitland daily mercury 13.11.1899,NaN,False,1,1825-01-01,24.0


### Interoperating with PySpark

In [31]:
shark_incidents_sdf = shark_incidents_df.to_spark(index_col="uin")
shark_incidents_sdf.printSchema()

shark_incidents_extended_sdf = shark_incidents_sdf.withColumn(
    "shark_length_ft", F.col("shark_length_m") * 3.28084
)

shark_incidents_extended_df = shark_incidents_extended_sdf.pandas_api(index_col="uin")
shark_incidents_extended_df.sort_values("shark_length_m", ascending=False).head()

root
 |-- uin: long (nullable = true)
 |-- incident_month: long (nullable = true)
 |-- incident_year: long (nullable = true)
 |-- victim_injury: string (nullable = true)
 |-- state: string (nullable = true)
 |-- location: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- site_category: string (nullable = true)
 |-- site_category_comment: string (nullable = true)
 |-- shark_common_name: string (nullable = true)
 |-- shark_scientific_name: string (nullable = true)
 |-- shark_identification_method: string (nullable = true)
 |-- shark_identification_source: string (nullable = true)
 |-- shark_length_m: double (nullable = true)
 |-- basis_for_length: string (nullable = true)
 |-- provoked_unprovoked: string (nullable = true)
 |-- provocative_act: string (nullable = true)
 |-- no_sharks: long (nullable = true)
 |-- victim_activity: string (nullable = true)
 |-- fish_speared_: string (nullable = true)
 |-- commercial_dive_activity: 

,incident_month,incident_year,victim_injury,state,location,latitude,longitude,site_category,site_category_comment,shark_common_name,shark_scientific_name,shark_identification_method,shark_identification_source,shark_length_m,basis_for_length,provoked_unprovoked,provocative_act,no_sharks,victim_activity,fish_speared_,commercial_dive_activity,object_of_bite,present_at_time_of_bite,direction_first_strike,shark_behaviour,victim_aware_of_shark,shark_captured,injury_location,injury_severity,victim_gender,victim_age,victim_clothing,clothing_coverage,dominant_clothing_colour,other_clothing_colour,clothing_pattern,fin_colour,diversionary_action_taken,diversionary_action_outcome,people_<3m,people_3_15m,time_of_incident,depth_of_incident_m,teeth_recovered,time_in_water_min,water_temperature_°c,total_water_depth_m,water_visability_m,distance_to_shore_m,spring_or_neap_tide,tidal_cycle,wind_condition,weather_condition,air_temperature_°c,personal_protective_device,deterrent_brand_and_type,data_source,reference,,was_fatal,incident_day,incident_date,length_rank,shark_length_ft
uin,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
345,2,1936,fatal,NSW,"manly, south steyne, sydney",-33.783333,151.283333,coastal,None,white shark,Carcharodon carcharias,"bite analysis, shark behaviour, geographical l...",None,6.0,witness/victim,unprovoked,None,1,swimming,None,None,None,swimming,None,None,N,None,other: body not recovered,None,male,14.0,None,None,None,None,None,None,None,None,NaN,1.0,1500.0,0.0,None,NaN,23.0,2.0,NaN,100.0,None,None,None,None,NaN,None,None,media outlet,"sharpe, green, shark&survl, the canberra times...",NaN,True,1,NaT,1.0,19.68504
596,2,1982,fatal,TAS,south cape bay,-43.566667,146.883333,coastal,None,white shark,Carcharodon carcharias,direct observation,None,6.0,"witness/victim, other: comparison to boat",provoked,None,1,spearfishing,None,None,None,"spearfishing, boating",None,bit victim on body,N,None,other: body not recovered,None,male,32.0,wetsuit/drysuit,full body,black,None,None,None,None,None,NaN,NaN,NaN,0.0,None,NaN,NaN,NaN,NaN,50.0,None,None,moderate breeze,None,NaN,speargun,None,media outlet,"green, shark&survl, c.black, daily mirror 1.3....",NaN,True,1,NaT,1.0,19.68504
606,6,1984,uninjured,TAS,cape raoul,-43.233333,147.783333,coastal,None,white shark,Carcharodon carcharias,direct observation,None,6.0,witness/victim,provoked,enticed shark,1,diving,None,None,Catch,catch,None,swam towards victim,Y,None,None,None,male,NaN,None,None,None,None,None,None,pushed at shark,None,NaN,NaN,NaN,NaN,None,NaN,NaN,5.0,NaN,NaN,None,None,None,None,NaN,None,None,media outlet,the mercury 16.6.1984,NaN,False,1,NaT,1.0,19.68504
609,10,1984,uninjured,TAS,"the lanterns, fortescue bay",-43.133333,148.000000,coastal,None,white shark,Carcharodon carcharias,direct observation,None,6.0,witness/victim,provoked,enticed shark,1,diving,None,Y,Catch,catch,None,circled victim,Y,None,None,None,male,NaN,wetsuit/drysuit,full body,black,None,None,None,hid in rock cave,None,NaN,NaN,NaN,23.0,None,5.0,NaN,NaN,23.0,NaN,None,None,None,None,NaN,None,None,media outlet,the mercury 15.10.1984,NaN,False,1,NaT,1.0,19.68504
613,3,1985,fatal,SA,"wiseman's beach, port lincoln",-34.466667,136.016667,coastal,None,white shark,Carcharodon carcharias,direct observation,None,6.0,witness/victim,unprovoked,None,1,snorkelling,None,None,None,snorkelling,behind,bit victim,N,None,torso,major lacerations,female,33.0,wetsuit/drysuit,full body,black,None,None,yellow,None,None,NaN,NaN,1230.0,NaN,None,NaN,NaN,NaN,NaN,150.0,None,None,None,None,NaN,None,None,media outlet,"papers, taylor.",NaN,True,1,NaT,1.0,19.68504


## Performance and Exporting

### Performance considerations

In [34]:
shark_incidents_sdf = shark_incidents_df.to_spark(index_col="uin")

# Caching underlying Spark DataFrame
shark_incidents_sdf.cache()

# Repartitoning for performance
shark_incidents_repartitioned_sdf = shark_incidents_sdf.repartition("shark_common_name")
shark_incidents_repartitioned_df = shark_incidents_repartitioned_sdf.pandas_api()

### Exporting Data

In [ ]:
shark_incidents_df.to_parquet("parquet/", index_col="uin")
shark_incidents_df.to_csv("csv/", index_col="uin")